# **Crypto On-chain Data Analysis**
This notebook analyzes historical tokenomics for major cryptocurrencies (BTC, ETH, BNB, SOL) using data from CoinGecko API.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pycoingecko import CoinGeckoAPI

# Initialize CoinGecko API client
cg = CoinGeckoAPI()

# Set default template for plotly
import plotly.io as pio
pio.templates.default = "plotly_dark"


In [2]:
# Define the cryptocurrencies we want to analyze
crypto = pd.DataFrame({
    'symbol': ['BTC', 'ETH', 'BNB', 'SOL', 'XRP', 'AVAX'],
    'name': ['Bitcoin', 'Ethereum', 'Binance Coin', 'Solana', 'XRP', 'Avalanche'],
    'coingecko_id': ['bitcoin', 'ethereum', 'binancecoin', 'solana', 'ripple', 'avalanche-2']
})

print("Cryptocurrencies to analyze:")
print(crypto)

Cryptocurrencies to analyze:
  symbol          name coingecko_id
0    BTC       Bitcoin      bitcoin
1    ETH      Ethereum     ethereum
2    BNB  Binance Coin  binancecoin
3    SOL        Solana       solana
4    XRP           XRP       ripple
5   AVAX     Avalanche  avalanche-2


In [3]:
# Fetching Tokenomics Data
tokenomics_data = []

for _, row in crypto.iterrows():
    try:
        print(f"Fetching data for {row['name']}...")
        # Fetch detailed coin data from CoinGecko
        coin_data = cg.get_coin_by_id(row['coingecko_id'])
        data = {
            'symbol': row['symbol'],
            'current_price': coin_data['market_data']['current_price']['usd'],
            'market_cap': coin_data['market_data']['market_cap']['usd'],
            'total_volume': coin_data['market_data']['total_volume']['usd'],
            'circulating_supply': coin_data['market_data']['circulating_supply'],
            'total_supply': coin_data['market_data']['total_supply'],
            'max_supply': coin_data['market_data']['max_supply'],
            'fdv': coin_data['market_data'].get('fully_diluted_valuation', {}).get('usd'),
        }
        tokenomics_data.append(data)
        
    except Exception as e:
        print(f"Error fetching data for {row['symbol']}: {e}")

# Convert to DataFrame
tokenomics_df = pd.DataFrame(tokenomics_data)

Fetching data for Bitcoin...
Fetching data for Ethereum...
Fetching data for Binance Coin...
Fetching data for Solana...
Fetching data for XRP...
Fetching data for Avalanche...


In [4]:
# 1. Market Cap Comparison (Bar Chart)
fig1 = px.bar(
    tokenomics_df,
    x='symbol',
    y='market_cap',
    title='<b>Market Capitalization by Cryptocurrency</b>',
    labels={'market_cap': 'Market Cap (USD)', 'symbol': 'Cryptocurrency'},
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
)

# Customize the layout
fig1.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
    yaxis_type='log',  # Logarithmic scale for better visibility
    yaxis=dict(
        type='log',
        tickvals=[1e10, 1e11, 1e12, 1e13],  # 10B, 100B, 1T, 10T
        ticktext=['$10B', '$100B', '$1T', '$10T'],
        showgrid=True
    )
    
)

# Show the plot
fig1.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [5]:
# 2. Supply Metrics Comparison (Grouped Bar Chart)
supply_data = pd.melt(
    tokenomics_df,
    id_vars=['symbol'],
    value_vars=['circulating_supply', 'total_supply', 'max_supply'],
    var_name='supply_type',
    value_name='supply',
)
fig2 = px.bar(
    supply_data,
    x='symbol',
    y='supply',
    color='supply_type',
    title='<b>Supply Metrics by Cryptocurrency</b>',
    barmode='group',
    labels={'supply': 'Supply Amount', 'symbol': 'Cryptocurrency'}
)

# Customize the layout
fig2.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
    yaxis=dict(
        type='log',
        tickvals=[1e6, 1e7, 1e8, 1e9, 1e10, 1e11],  # 1M, 10M, 100M, 1B, 10B, 100B
        ticktext=['1M', '10M', '100M', '1B', '10B', '100B'],
        showgrid=True
    ),
    
)

fig2.show()

In [6]:
# 3. Market Cap vs Volume (Scatter Plot)
fig3 = px.scatter(
    tokenomics_df,
    x='market_cap',
    y='total_volume',
    text='symbol',
    title='<b>Market Cap vs Trading Volume</b>',
    labels={
        'market_cap': 'Market Cap (USD)',
        'total_volume': 'Trading Volume (USD)'
    },
    size='market_cap',
    size_max=150,
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
    log_x=True,
    log_y=True
)

# Customize the layout
fig3.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig3.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [7]:
# 4. Price vs Supply (Bubble Chart)
fig4 = px.scatter(
    tokenomics_df,
    x='circulating_supply',
    y='current_price',
    size='market_cap',
    size_max=150,
    text='symbol',
    title='<b>Price vs Circulating Supply</b>',
    subtitle='Bubble size = Market Cap',
    labels={
        'circulating_supply': 'Circulating Supply',
        'current_price': 'Price (USD)',
    },
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
    log_x=True,
    log_y=True
)

# Customize the layout
fig4.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig4.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [8]:
# 5. Market Dominance (Pie Chart)
fig5 = px.pie(
    tokenomics_df,
    values='market_cap',
    names='symbol',
    title='<b>Market Cap Distribution</b>',
    subtitle='From Selected Cryptocurrencies',
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
)

# Customize the layout
fig5.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig5.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [81]:
# Update the format_number function to include trillions
def format_number(value):
    """Format large numbers to abbreviated string with T, B, M"""
    if value >= 1e12:
        return f'${value/1e12:.1f}T'
    elif value >= 1e9:
        return f'${value/1e9:.1f}B'
    elif value >= 1e6:
        return f'${value/1e6:.1f}M'
    else:
        return f'${value:.0f}'

# Format the values in the DataFrame
tokenomics_df['fdv_formatted'] = tokenomics_df['fdv'].apply(format_number)
tokenomics_df['volume_formatted'] = tokenomics_df['total_volume'].apply(format_number)
tokenomics_df['price_formatted'] = tokenomics_df['current_price'].apply(lambda x: f'${x:,.2f}')
tokenomics_df['market_cap_formatted'] = tokenomics_df['market_cap'].apply(format_number, lambda x: f'${x:,.2f}')

C:\Users\Matheus\AppData\Local\Temp\ipykernel_15648\2179181371.py:17: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.



In [ ]:
# Create 3D scatter plot
fig6 = px.scatter_3d(
    tokenomics_df,
    x='market_cap',
    y='total_volume',
    z='fdv',
    color='symbol',
    size='fdv',
    size_max=80,
    title='<b>Crypto Market Metrics - 3D View</b>',
    labels={
        'fdv': 'Fully Diluted Valuation',
        'total_volume': 'Volume',
        'market_cap': 'Market Cap'
    },
    color_discrete_map={
        'BTC': '#F7931A',
        'ETH': '#636BFF',
        'BNB': '#F3BA2F',
        'SOL': '#CF62E7',
        'XRP': '#4B8BBE',
        'AVAX': '#E84142'
    },
    custom_data=['symbol', 'fdv_formatted', 'volume_formatted', 'market_cap_formatted']  # Add formatted values    
)



# Update hover template
fig6.update_traces(
    hovertemplate="<br>".join([
        "<b>%{customdata[0]}</b>",  # Crypto name
        "<b>%{customdata[1]}</b> FDV",
        "<b>%{customdata[2]}</b> Volume",
        "<b>%{customdata[3]}</b> Market Cap",
        "<extra></extra>"
    ])
)

# Customize the layout
fig6.update_layout(
    template='plotly_dark',
    scene=dict(
        xaxis=dict(
            type='log',
            showticklabels=False
        ),
        yaxis=dict(
            type='log',
            showticklabels=False
        ),
        zaxis=dict(
            type='log',
            showticklabels=False
        ),
    ),
    font_family="IBM Plex Sans",
    height=800/1.25,  # Adjust height
    width=800/1.25,  # Adjust width
    scene_camera=dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1, y=-2, z=1)
    )
)

# Show the plot
fig6.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
fig6.write_html("plots/html/3dview_crypto_market_metrics_.html")